# Command Pattern Tutorial 🎮

## What is the Command Pattern?

The Command Pattern encapsulates a request as an object, thereby letting you parameterize clients with different requests, queue operations, and support undo operations. Think of it as turning actions into objects.

**Real-world analogy**: Consider a remote control. Each button is a command object that knows how to perform a specific action (turn on TV, change channel, adjust volume). The remote doesn't need to know how these actions work internally - it just executes the commands when buttons are pressed.

## Table of Contents
1. [What is the Command Pattern?](#what-is-the-command-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Macro Commands and Queuing (Advanced)](#advanced-example-macro-commands-and-queuing)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## Why Do We Need It?

Let's see what happens when we tightly couple the invoker with the receiver:

In [ ]:
# Problem: Tight coupling between invoker and receiver

class Light:
    def __init__(self, location):
        self.location = location
        self.is_on = False
    
    def turn_on(self):
        self.is_on = True
        print(f"{self.location} light is ON")
    
    def turn_off(self):
        self.is_on = False
        print(f"{self.location} light is OFF")

class Fan:
    def __init__(self, location):
        self.location = location
        self.speed = 0
    
    def high_speed(self):
        self.speed = 3
        print(f"{self.location} fan is on HIGH speed")
    
    def off(self):
        self.speed = 0
        print(f"{self.location} fan is OFF")

# Problematic remote control
class BadRemoteControl:
    def __init__(self):
        self.light = None
        self.fan = None
    
    def set_light(self, light):
        self.light = light
    
    def set_fan(self, fan):
        self.fan = fan
    
    def button_1_pressed(self):
        if self.light:
            self.light.turn_on()  # Tightly coupled!
    
    def button_2_pressed(self):
        if self.light:
            self.light.turn_off()  # Hard to change behavior!
    
    def button_3_pressed(self):
        if self.fan:
            self.fan.high_speed()  # What if we want different behavior?

# Problems:
# 1. Remote control needs to know about all device types
# 2. Hard to add new devices or change button behavior
# 3. No way to undo operations
# 4. Can't queue or log operations

## Simple Implementation

Let's solve this with the Command Pattern:

In [ ]:
from abc import ABC, abstractmethod

# Command interface
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass
    
    @abstractmethod
    def undo(self):
        pass

# Receivers (the objects that know how to perform the work)
class Light:
    def __init__(self, location):
        self.location = location
        self.is_on = False
    
    def turn_on(self):
        self.is_on = True
        print(f"{self.location} light is ON")
    
    def turn_off(self):
        self.is_on = False
        print(f"{self.location} light is OFF")

class Fan:
    def __init__(self, location):
        self.location = location
        self.speed = 0
    
    def high_speed(self):
        self.speed = 3
        print(f"{self.location} fan is on HIGH speed")
    
    def medium_speed(self):
        self.speed = 2
        print(f"{self.location} fan is on MEDIUM speed")
    
    def off(self):
        self.speed = 0
        print(f"{self.location} fan is OFF")

# Concrete commands
class LightOnCommand(Command):
    def __init__(self, light: Light):
        self.light = light
    
    def execute(self):
        self.light.turn_on()
    
    def undo(self):
        self.light.turn_off()

class LightOffCommand(Command):
    def __init__(self, light: Light):
        self.light = light
    
    def execute(self):
        self.light.turn_off()
    
    def undo(self):
        self.light.turn_on()

class FanHighCommand(Command):
    def __init__(self, fan: Fan):
        self.fan = fan
        self.prev_speed = 0
    
    def execute(self):
        self.prev_speed = self.fan.speed
        self.fan.high_speed()
    
    def undo(self):
        if self.prev_speed == 3:
            self.fan.high_speed()
        elif self.prev_speed == 2:
            self.fan.medium_speed()
        else:
            self.fan.off()

# Null Object pattern for empty slots
class NoCommand(Command):
    def execute(self):
        pass
    
    def undo(self):
        pass

In [ ]:
# Invoker
class RemoteControl:
    def __init__(self):
        # Initialize with no-op commands
        no_command = NoCommand()
        self.on_commands = [no_command] * 7
        self.off_commands = [no_command] * 7
        self.undo_command = no_command
    
    def set_command(self, slot, on_command, off_command):
        self.on_commands[slot] = on_command
        self.off_commands[slot] = off_command
    
    def on_button_pressed(self, slot):
        self.on_commands[slot].execute()
        self.undo_command = self.on_commands[slot]
    
    def off_button_pressed(self, slot):
        self.off_commands[slot].execute()
        self.undo_command = self.off_commands[slot]
    
    def undo_button_pressed(self):
        self.undo_command.undo()

# Using the remote control
remote = RemoteControl()

# Create devices
living_room_light = Light("Living Room")
kitchen_light = Light("Kitchen")
ceiling_fan = Fan("Ceiling")

# Create commands
living_room_light_on = LightOnCommand(living_room_light)
living_room_light_off = LightOffCommand(living_room_light)
kitchen_light_on = LightOnCommand(kitchen_light)
kitchen_light_off = LightOffCommand(kitchen_light)
ceiling_fan_high = FanHighCommand(ceiling_fan)
ceiling_fan_off = LightOffCommand(ceiling_fan)  # Reusing command!

# Set commands to remote slots
remote.set_command(0, living_room_light_on, living_room_light_off)
remote.set_command(1, kitchen_light_on, kitchen_light_off)
remote.set_command(2, ceiling_fan_high, ceiling_fan_off)

# Test the remote
print("--- Testing Remote Control ---")
remote.on_button_pressed(0)   # Turn on living room light
remote.off_button_pressed(0)  # Turn off living room light
remote.undo_button_pressed()  # Undo (turn on again)

remote.on_button_pressed(2)   # Turn on ceiling fan
remote.undo_button_pressed()  # Undo (turn off fan)

## Understanding the Implementation

### Key Concepts:

1. **Command**: Declares an interface for executing operations
2. **Concrete Command**: Implements the Command interface and invokes corresponding operations on Receiver
3. **Receiver**: Knows how to perform the operations
4. **Invoker**: Asks the command to carry out the request
5. **Client**: Creates a ConcreteCommand object and sets its receiver

### Benefits:
- **Decoupling**: Invoker doesn't need to know about receivers
- **Flexibility**: Easy to add new commands without changing existing code
- **Undo/Redo**: Commands can store state for undoing operations
- **Macro commands**: Combine multiple commands
- **Queuing and logging**: Commands can be stored and executed later

## Advanced Example: Macro Commands and Queuing

Let's create more advanced features like macro commands and command queuing:

In [ ]:
# Macro command - combines multiple commands
class MacroCommand(Command):
    def __init__(self, commands):
        self.commands = commands
    
    def execute(self):
        for command in self.commands:
            command.execute()
    
    def undo(self):
        # Undo in reverse order
        for command in reversed(self.commands):
            command.undo()

# Command queue for delayed execution
class CommandQueue:
    def __init__(self):
        self.commands = []
    
    def add_command(self, command):
        self.commands.append(command)
        print(f"Command queued. Queue size: {len(self.commands)}")
    
    def execute_all(self):
        print("Executing all queued commands...")
        for command in self.commands:
            command.execute()
        self.commands.clear()
        print("All commands executed. Queue cleared.")

# Additional device for demonstration
class Stereo:
    def __init__(self, location):
        self.location = location
        self.is_on = False
        self.volume = 0
    
    def on(self):
        self.is_on = True
        print(f"{self.location} stereo is ON")
    
    def off(self):
        self.is_on = False
        print(f"{self.location} stereo is OFF")
    
    def set_volume(self, volume):
        self.volume = volume
        print(f"{self.location} stereo volume set to {volume}")

class StereoOnWithVolumeCommand(Command):
    def __init__(self, stereo: Stereo, volume: int = 11):
        self.stereo = stereo
        self.volume = volume
        self.prev_volume = 0
    
    def execute(self):
        self.prev_volume = self.stereo.volume
        self.stereo.on()
        self.stereo.set_volume(self.volume)
    
    def undo(self):
        self.stereo.set_volume(self.prev_volume)
        self.stereo.off()

# Create devices and commands
living_room_light = Light("Living Room")
living_room_stereo = Stereo("Living Room")
ceiling_fan = Fan("Ceiling")

light_on = LightOnCommand(living_room_light)
stereo_on = StereoOnWithVolumeCommand(living_room_stereo, 11)
fan_on = FanHighCommand(ceiling_fan)

# Create a "party mode" macro command
party_mode_on = MacroCommand([light_on, stereo_on, fan_on])

print("--- Testing Macro Command ---")
party_mode_on.execute()
print("\n--- Undoing Party Mode ---")
party_mode_on.undo()

In [ ]:
# Testing command queue
print("\n--- Testing Command Queue ---")
queue = CommandQueue()

# Add commands to queue
queue.add_command(LightOnCommand(living_room_light))
queue.add_command(StereoOnWithVolumeCommand(living_room_stereo, 5))
queue.add_command(FanHighCommand(ceiling_fan))

# Execute all at once
queue.execute_all()

## Real-world Example: Text Editor with Undo/Redo

Let's implement a simple text editor that supports undo/redo operations:

In [ ]:
class TextEditor:
    def __init__(self):
        self.content = ""
        self.history = []
        self.current_position = -1
    
    def execute_command(self, command):
        # Remove any commands after current position (for redo functionality)
        self.history = self.history[:self.current_position + 1]
        
        # Execute and add to history
        command.execute()
        self.history.append(command)
        self.current_position += 1
    
    def undo(self):
        if self.current_position >= 0:
            command = self.history[self.current_position]
            command.undo()
            self.current_position -= 1
            print("Undo performed")
        else:
            print("Nothing to undo")
    
    def redo(self):
        if self.current_position < len(self.history) - 1:
            self.current_position += 1
            command = self.history[self.current_position]
            command.execute()
            print("Redo performed")
        else:
            print("Nothing to redo")
    
    def get_content(self):
        return self.content
    
    def set_content(self, content):
        self.content = content

class InsertTextCommand(Command):
    def __init__(self, editor: TextEditor, text: str, position: int):
        self.editor = editor
        self.text = text
        self.position = position
    
    def execute(self):
        content = self.editor.get_content()
        new_content = content[:self.position] + self.text + content[self.position:]
        self.editor.set_content(new_content)
        print(f"Inserted '{self.text}' at position {self.position}")
    
    def undo(self):
        content = self.editor.get_content()
        new_content = content[:self.position] + content[self.position + len(self.text):]
        self.editor.set_content(new_content)
        print(f"Removed '{self.text}' from position {self.position}")

class DeleteTextCommand(Command):
    def __init__(self, editor: TextEditor, start: int, length: int):
        self.editor = editor
        self.start = start
        self.length = length
        self.deleted_text = ""
    
    def execute(self):
        content = self.editor.get_content()
        self.deleted_text = content[self.start:self.start + self.length]
        new_content = content[:self.start] + content[self.start + self.length:]
        self.editor.set_content(new_content)
        print(f"Deleted '{self.deleted_text}' from position {self.start}")
    
    def undo(self):
        content = self.editor.get_content()
        new_content = content[:self.start] + self.deleted_text + content[self.start:]
        self.editor.set_content(new_content)
        print(f"Restored '{self.deleted_text}' at position {self.start}")

# Testing the text editor
editor = TextEditor()

print("--- Text Editor with Undo/Redo ---")
print(f"Initial content: '{editor.get_content()}'")

# Insert some text
editor.execute_command(InsertTextCommand(editor, "Hello", 0))
print(f"Content: '{editor.get_content()}'")

editor.execute_command(InsertTextCommand(editor, " World", 5))
print(f"Content: '{editor.get_content()}'")

editor.execute_command(InsertTextCommand(editor, "!", 11))
print(f"Content: '{editor.get_content()}'")

# Test undo
editor.undo()
print(f"After undo: '{editor.get_content()}'")

editor.undo()
print(f"After undo: '{editor.get_content()}'")

# Test redo
editor.redo()
print(f"After redo: '{editor.get_content()}'")

# Test delete
editor.execute_command(DeleteTextCommand(editor, 0, 5))  # Delete "Hello"
print(f"After delete: '{editor.get_content()}'")

editor.undo()
print(f"After undo delete: '{editor.get_content()}'")

## When to Use (and When NOT to Use)

### Use Command Pattern when:
- You want to parameterize objects with operations
- You want to queue operations, schedule their execution, or execute them remotely
- You want to support undo operations
- You want to log changes so that they can be reapplied in case of a system crash
- You want to structure a system around high-level operations built on primitive operations

### Don't use when:
- The operations are simple and don't benefit from the extra abstraction
- You don't need undo functionality or operation queuing
- The overhead of creating command objects is significant

### Real-world applications:
- GUI buttons and menu items
- Macro recording in applications
- Database transactions
- Network request queuing
- Game action systems (moves, attacks, spells)
- Web server request handling
- Job schedulers and task queues